In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Datasets/Melbourne Housing/melb_data.csv')
data.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
5872,St Kilda,13/62 Wellington St,1,u,461000.0,SP,McGrath,27/11/2016,6.1,3182.0,...,1.0,1.0,0.0,NaN,NaN,Port Phillip,-37.8571,144.9870,Southern Metropolitan,13240.0
8367,South Yarra,1/52 Caroline St,1,u,410000.0,S,hockingstuart,8/04/2017,3.3,3141.0,...,1.0,0.0,0.0,NaN,NaN,Stonnington,-37.8378,144.9883,Southern Metropolitan,14887.0
5775,St Kilda,59 Clyde St,3,h,1891000.0,S,Marshall,4/03/2017,6.1,3182.0,...,2.0,3.0,384.0,123.0,1915.0,Port Phillip,-37.8645,144.9805,Southern Metropolitan,13240.0
8689,Toorak,4/5 Selwyn Ct,3,u,922500.0,S,Rodney,29/04/2017,4.6,3142.0,...,1.0,2.0,4657.0,NaN,1980.0,Stonnington,-37.8421,145.0237,Southern Metropolitan,7217.0
1384,Brunswick,10 Cliff St,1,h,772000.0,SP,Collins,28/08/2016,5.2,3056.0,...,1.0,1.0,147.0,82.0,1890.0,Moreland,-37.7643,144.9584,Northern Metropolitan,11918.0


In [3]:
# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

In [4]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [5]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

In [6]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [7]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [8]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


### Step 1: Define Preprocessing Steps¶
`ColumnTransformer` class to bundle together different preprocessing steps. :

* imputes missing values in numerical data, and
* imputes missing values and applies a one-hot encoding to categorical data.

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [10]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

In [11]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [12]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Step 2: Define the Model

In [13]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [14]:
from sklearn.metrics import mean_absolute_error

In [15]:
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [16]:
# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Type', 'Method',
                                                   'Regionname'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [17]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

In [18]:
# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855
